## **앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


- 직접 투표는 개별 분류기의 예측 결과 중 다수결 원칙으로 최종 클래스를 결정하며, 모든 분류기에서 사용 가능한 반면, 간접 투표는 각 분류기가 예측한 확률을 평균 내어 가장 높은 확률을 가진 클래스를 선택하며, predict_proba()를 지원하는 모델에서만 가능

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

- 학습률 낮추기, 트리 개수 줄이고 깊이 제한하기

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [3]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=42, test_size=10000)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [5]:
# model fitting
# 시간 너무 오래걸려서 모델 파라미터 수정하여 n_estimators 낮춤
rf_clf = RandomForestClassifier(n_estimators=30, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=30, random_state=42)
svm_clf = LinearSVC(random_state=30, max_iter=100)
mlp_clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=100, random_state=42)


# 모델 학습
rf_clf.fit(X_train, y_train)
et_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(50,), max_iter=100, random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [6]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('et', et_clf),
        ('svm', svm_clf),
        ('mlp', mlp_clf)
    ],
    voting='hard'
)

In [7]:
# model fitting
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=30,
                                                     random_state=42)),
                             ('et',
                              ExtraTreesClassifier(n_estimators=30,
                                                   random_state=42)),
                             ('svm', LinearSVC(max_iter=100, random_state=30)),
                             ('mlp',
                              MLPClassifier(hidden_layer_sizes=(50,),
                                            max_iter=100, random_state=42))])

In [8]:
# model test
score = voting_clf.score(X_val, y_val)
print(score)

0.9661333333333333


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [9]:
# 각 분류 모델 학습
rf_score = rf_clf.score(X_val, y_val)
et_score = et_clf.score(X_val, y_val)
svm_score = svm_clf.score(X_val, y_val)
mlp_score = mlp_clf.score(X_val, y_val)

In [10]:
# 각 분류 모델의 성능 확인
print(f"랜덤포레스트 분류기 정확도: {rf_score:.4f}")
print(f"엑스트라 트리 분류기 정확도: {et_score:.4f}")
print(f"SVM 분류기 정확도: {svm_score:.4f}")
print(f"MLP 분류기 정확도: {mlp_score:.4f}")


랜덤포레스트 분류기 정확도: 0.9639
엑스트라 트리 분류기 정확도: 0.9665
SVM 분류기 정확도: 0.9089
MLP 분류기 정확도: 0.9533


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. SVM 모델

In [11]:
# 가장 성능이 낮은 모델 제거
model_scores = {
    'rf': rf_score,
    'et': et_score,
    'svm': svm_score,
    'mlp': mlp_score
}
lowest_model = min(model_scores, key=model_scores.get)
del model_scores[lowest_model]

In [12]:
# model fitting
estimators_updated = [(name, clf) for name, clf in [
    ('rf', rf_clf),
    ('et', et_clf),
    ('mlp', mlp_clf)
] if name != lowest_model]  # 가장 낮은 모델 제거

voting_clf_updated = VotingClassifier(estimators=estimators_updated, voting='hard')

voting_clf_updated.fit(X_train, y_train)


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=30,
                                                     random_state=42)),
                             ('et',
                              ExtraTreesClassifier(n_estimators=30,
                                                   random_state=42)),
                             ('mlp',
                              MLPClassifier(hidden_layer_sizes=(50,),
                                            max_iter=100, random_state=42))])

In [ ]:
# 모델 제거 후 성능 확인
updated_score = voting_clf_updated.score(X_val, y_val)

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [13]:
# 각 모델을 훈련시킨 후 검증 세트에 대한 예측을 만듦
rf_pred = rf_clf.predict(X_val)
et_pred = et_clf.predict(X_val)
svm_pred = svm_clf.predict(X_val)
mlp_pred = mlp_clf.predict(X_val)

X_meta = np.column_stack((rf_pred, et_pred, svm_pred, mlp_pred))
y_meta = y_val  # 검증 세트의 실제 값


####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [14]:
from sklearn.ensemble import RandomForestClassifier
meta_clf = RandomForestClassifier(random_state=42)
# 모델 훈련
meta_clf.fit(X_meta, y_meta)


RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [15]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
# 각 분류 모델의 테스트 세트 예측
rf_test_pred = rf_clf.predict(X_test)
et_test_pred = et_clf.predict(X_test)
svm_test_pred = svm_clf.predict(X_test)
mlp_test_pred = mlp_clf.predict(X_test)

# 예측값을 메타 모델(블렌더)로 입력할 수 있도록 새로운 데이터셋으로 만듦
X_test_meta = np.column_stack((rf_test_pred, et_test_pred, svm_test_pred, mlp_test_pred))

In [16]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
final_predictions = meta_clf.predict(X_test_meta)


In [17]:
# model test
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, final_predictions)
print(f"스태킹 앙상블 모델의 정확도: {accuracy:.4f}")

스태킹 앙상블 모델의 정확도: 0.9630
